In [ ]:
# Imports
import numpy as np
import pandas as pd

import counter as counter

In [ ]:
# File path to the images
data_dir = 'Lung_Paper_Images/Raw_Images/Lung/'

size_thresh = 0.62,
gaussian_size = 5,
truncation = 2,
threshold = 1000,
min_size = 10

# Initialize a dataframe
df = pd.DataFrame(columns=['Date', 
                           'Size Threshold', 
                           'Gaussian Size', 
                           'Truncation', 
                           'Intensity Threshold',
                           'Minimum Size',
                           'Applied Threshold',
                           'Virus', 
                           'Animal', 
                           'Replicate',  
                           'Count',
                           'Cells Quantified',
                           'Brightness List',
                           'Applied Thresh',
                           'Image Multiplication Factor',
                           'Minimum Pixel Value',
                           'Maximum Pixel Value'])

#df = pd.read_csv('Lung_Paper_Images/Post-Quantification/Brain/20190917_Stitched_Brain_Cell_Counts_with_Brightness_sampled.csv', comment='#')

# Loop through all of the images 
for virus in ['AAV5', 'AAV9', 'CAPA4']:
    for animal in ['1', '2', '3', '4', '5', '6']:
        for replicate in ['1', '2']:
            # Initialize the filename and read in the image
            fname = data_dir + virus + '/' + animal + '/' + replicate +'/'
            
            n_cells, cell_list, cell_intensity_list, actual_thresh, mult, min_v, max_v = counter.cell_counter(fname,
                                                                                                              size_thresh = 0.62,
                                                                                                              gaussian_size = 5,
                                                                                                              truncation = 2,
                                                                                                              threshold = 1000,
                                                                                                              min_size = 10)
            
            # Write all the information into a tidy dataframe
            df = df.append({'Date' : '2020-06-12',
                            'Size Threshold' : size_thresh,
                            'Gaussian Size' : gaussian_size,
                            'Truncation' : truncation,
                            'Intensity Threshold' : threshold, 
                            'Minimum Size' : min_size,
                            'Virus' : virus, 
                            'Animal' : animal, 
                            'Replicate' : replicate, 
                            'Count' : int(n_cells),
                            'Cells Quantified' : cell_list, 
                            'Brightness List': cell_intensity_list, 
                            'Applied Thresh': actual_thresh,
                            'Image Multiplication Factor': mult, 
                            'Minimum Pixel Value': min_v,
                            'Maximum Pixel Value': max_v},
                           ignore_index=True)
            
            # Save the dataframe
            df.to_csv('Lung_Paper_Images/Post-Quantification/Lung/20200612_Stitched_Lung_Cell_Counts_with_Brightness_sampled.csv', index=False)